In [ ]:
%pip install pykrx

In [ ]:
from datetime import datetime, timedelta
datetime.now()

datetime.datetime(2025, 6, 22, 10, 22, 22, 211577)

In [22]:
datetime.now() - timedelta(days=3)

datetime.datetime(2025, 6, 19, 10, 23, 20, 773943)

In [23]:
from datetime import datetime, timedelta

def get_target_stock_date(today: datetime) -> datetime:
    weekday = today.weekday()  # 0=월, ..., 6=일
    now = datetime.now()

    if weekday == 0 and now.hour < 15:  # 월요일 + 마감 전
        return today - timedelta(days=3)  # 이전 주 금요일
    elif weekday in [1, 2, 3, 4] and now.hour < 15:  # 화~금 + 마감 전
        return today - timedelta(days=1)  # 어제
    elif weekday in [0, 1, 2, 3, 4] and now.hour >= 15:  # 월~금 + 마감 후
        return today  # 오늘
    elif weekday == 5:  # 토요일
        return today - timedelta(days=1)  # 금요일
    elif weekday == 6:  # 일요일
        return today - timedelta(days=2)  # 금요일
    else:
        raise ValueError("날짜 판단 오류")

In [30]:
from pykrx import stock
stock_list = stock.get_market_ticker_list(market="ALL")
stock_list[:3]

['060310', '095570', '006840']

In [31]:
stock_name = '삼성전자'
for tiker in stock_list:
    if stock_name == stock.get_market_ticker_name(tiker) :
        print(tiker)
        break

005930


In [34]:
target_str = '2025-06-20'

In [40]:
df = stock.get_market_ohlcv_by_date(fromdate=target_str, todate=target_str, ticker='005930')
row = df.iloc[0]
row

시가     5.940000e+04
고가     5.980000e+04
저가     5.910000e+04
종가     5.950000e+04
거래량    1.718480e+07
등락률    5.067568e-01
Name: 2025-06-20 00:00:00, dtype: float64

In [42]:
from pykrx import stock
from datetime import datetime, timedelta

def get_target_stock_date(today: datetime) -> datetime:
    weekday = today.weekday()
    now = datetime.now()

    if weekday == 0 and now.hour < 15:  # 월요일 + 마감 전
        return today - timedelta(days=3)
    elif weekday in [1, 2, 3, 4] and now.hour < 15:  # 화~금 + 마감 전
        return today - timedelta(days=1)
    elif weekday in [0, 1, 2, 3, 4] and now.hour >= 15:  # 월~금 + 마감 후
        return today
    elif weekday == 5:  # 토요일
        return today - timedelta(days=1)
    elif weekday == 6:  # 일요일
        return today - timedelta(days=2)
    else:
        raise ValueError("날짜 판단 오류")

def get_stock_code_by_name(name: str) -> str:
    tickers = stock.get_market_ticker_list(market="ALL")
    for ticker in tickers:
        if stock.get_market_ticker_name(ticker) == name:
            return ticker
    raise ValueError(f"종목명 '{name}'을 찾을 수 없습니다.")

def get_korean_stock_info(stock_name: str):
    try:
        code = get_stock_code_by_name(stock_name)
        today = datetime.today()
        target_date = get_target_stock_date(today)
        target_str = target_date.strftime("%Y%m%d")

        df = stock.get_market_ohlcv_by_date(fromdate=target_str, todate=target_str, ticker=code)
        if df.empty:
            print("해당 날짜에 데이터가 없습니다.")
            return

        row = df.iloc[0]
        print(f"[{stock_name}] {target_str} 기준")
        print(f"종가: {row['종가']:,}원")
        print(f"전일대비: {row['등락률']}%")
        print(f"거래량: {row['거래량']:,}주")
    except Exception as e:
        print("오류:", e)

# 사용 예시
get_korean_stock_info("삼성")


오류: 종목명 '삼성'을 찾을 수 없습니다.


In [45]:
matched = {}
matched['삼성전자'] = '001930'
matched['삼성전기'] = '0009150'
matched['삼성SDI'] = '006400'
matched['하이닉스'] = '000660'

In [49]:
for i in matched.items():
    print(i)


('삼성전자', '001930')
('삼성전기', '0009150')
('삼성SDI', '006400')
('하이닉스', '000660')


In [50]:
for i in enumerate(matched.items(),1):
    print(i)

(1, ('삼성전자', '001930'))
(2, ('삼성전기', '0009150'))
(3, ('삼성SDI', '006400'))
(4, ('하이닉스', '000660'))


In [53]:
'삼성' in '삼성전기'

True

In [54]:
from pykrx import stock
from datetime import datetime, timedelta

def get_target_stock_date(today: datetime) -> datetime:
    weekday = today.weekday()
    now = datetime.now()

    if weekday == 0 and now.hour < 15:  # 월요일 + 마감 전
        return today - timedelta(days=3)
    elif weekday in [1, 2, 3, 4] and now.hour < 15:  # 화~금 + 마감 전
        return today - timedelta(days=1)
    elif weekday in [0, 1, 2, 3, 4] and now.hour >= 15:  # 월~금 + 마감 후
        return today
    elif weekday == 5:  # 토요일
        return today - timedelta(days=1)
    elif weekday == 6:  # 일요일
        return today - timedelta(days=2)
    else:
        raise ValueError("날짜 판단 오류")

def get_stock_code_by_name(name: str) -> str:
    tickers = stock.get_market_ticker_list(market="ALL")
    matched = {}

    for ticker in tickers:
        kor_name = stock.get_market_ticker_name(ticker)
        if kor_name == name:
            return ticker
        if name in kor_name:
            matched[kor_name] = ticker

    if matched:
        print(f"'{name}'(으)로 검색된 유사 종목 목록:")
        for i, (k, v) in enumerate(matched.items(), 1):
            print(f"{i}. {k} ({v})")
        raise ValueError(f"'{name}'와 정확히 일치하는 종목은 없지만, 위의 유사 종목이 있습니다.")
    else:
        raise ValueError(f"종목명 '{name}'을 찾을 수 없고, 유사한 이름도 없습니다.")

def get_korean_stock_info(stock_name: str):
    try:
        code = get_stock_code_by_name(stock_name)
        today = datetime.today()
        target_date = get_target_stock_date(today)
        target_str = target_date.strftime("%Y%m%d")

        df = stock.get_market_ohlcv_by_date(fromdate=target_str, todate=target_str, ticker=code)
        if df.empty:
            print("해당 날짜에 데이터가 없습니다.")
            return

        row = df.iloc[0]
        print(f"[{stock_name}] {target_str} 기준")
        print(f"종가: {row['종가']:,}원")
        print(f"전일대비: {row['등락률']}%")
        print(f"거래량: {row['거래량']:,}주")
    except Exception as e:
        print("오류:", e)

# 예시 사용
get_korean_stock_info("삼성")


'삼성'(으)로 검색된 유사 종목 목록:
1. 삼성E&A (028050)
2. 삼성FN리츠 (448730)
3. 삼성SDI (006400)
4. 삼성SDI우 (006405)
5. 삼성공조 (006660)
6. 삼성물산 (028260)
7. 삼성물산우B (02826K)
8. 삼성바이오로직스 (207940)
9. 삼성생명 (032830)
10. 삼성스팩8호 (448740)
11. 삼성에스디에스 (018260)
12. 삼성전기 (009150)
13. 삼성전기우 (009155)
14. 삼성전자 (005930)
15. 삼성전자우 (005935)
16. 삼성제약 (001360)
17. 삼성중공업 (010140)
18. 삼성증권 (016360)
19. 삼성출판사 (068290)
20. 삼성카드 (029780)
21. 삼성화재 (000810)
22. 삼성화재우 (000815)
오류: '삼성'와 정확히 일치하는 종목은 없지만, 위의 유사 종목이 있습니다.


In [57]:
get_korean_stock_info("SK하이닉스")

[SK하이닉스] 20250620 기준
종가: 257,000.0원
전일대비: 4.471544715447155%
거래량: 3,548,848.0주


# 커서가 수정해준 내용

In [5]:
from pykrx import stock
from datetime import datetime, timedelta
from typing import Optional, Dict, Any
import logging

# 상수 정의
MARKET_CLOSE_HOUR = 15
WEEKDAYS = [0, 1, 2, 3, 4]  # 월~금
WEEKEND = [5, 6]  # 토, 일

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 종목 코드 캐시
_stock_code_cache: Dict[str, str] = {}

def get_target_stock_date(today: datetime) -> datetime:
    """
    주어진 날짜를 기준으로 주식 데이터를 조회할 대상 날짜를 계산합니다.
    
    Args:
        today: 기준 날짜
        
    Returns:
        조회할 대상 날짜
        
    Raises:
        ValueError: 날짜 판단이 불가능한 경우
    """
    weekday = today.weekday()
    current_hour = today.hour

    if weekday == 0 and current_hour < MARKET_CLOSE_HOUR:  # 월요일 + 마감 전
        return today - timedelta(days=3)
    elif weekday in [1, 2, 3, 4] and current_hour < MARKET_CLOSE_HOUR:  # 화~금 + 마감 전
        return today - timedelta(days=1)
    elif weekday in WEEKDAYS and current_hour >= MARKET_CLOSE_HOUR:  # 월~금 + 마감 후
        return today
    elif weekday == 5:  # 토요일
        return today - timedelta(days=1)
    elif weekday == 6:  # 일요일
        return today - timedelta(days=2)
    else:
        raise ValueError("날짜 판단 오류")

def get_stock_code_by_name(name: str) -> str:
    """
    종목명으로 종목 코드를 조회합니다. 캐시를 사용하여 성능을 개선합니다.
    
    Args:
        name: 종목명
        
    Returns:
        종목 코드
        
    Raises:
        ValueError: 종목명을 찾을 수 없는 경우
    """
    # 캐시에서 먼저 확인
    if name in _stock_code_cache:
        return _stock_code_cache[name]
    
    try:
        tickers = stock.get_market_ticker_list(market="ALL")
        for ticker in tickers:
            ticker_name = stock.get_market_ticker_name(ticker)
            if ticker_name == name:
                # 캐시에 저장
                _stock_code_cache[name] = ticker
                return ticker
        raise ValueError(f"종목명 '{name}'을 찾을 수 없습니다.")
    except Exception as e:
        logger.error(f"종목 코드 조회 중 오류 발생: {e}")
        raise

def format_stock_data(row: Any, stock_name: str, target_str: str) -> str:
    """
    주식 데이터를 포맷팅하여 문자열로 반환합니다.
    
    Args:
        row: 주식 데이터 행
        stock_name: 종목명
        target_str: 대상 날짜 문자열
        
    Returns:
        포맷팅된 주식 정보 문자열
    """
    try:
        return (
            f"[{stock_name}] {target_str} 기준\n"
            f"종가: {row['종가']:,}원\n"
            f"전일대비: {row['등락률']:.2f}%\n"
            f"거래량: {row['거래량']:,}주\n"
            f"시가: {row['시가']:,}원\n"
            f"고가: {row['고가']:,}원\n"
            f"저가: {row['저가']:,}원"
        )
    except KeyError as e:
        logger.error(f"주식 데이터 포맷팅 중 누락된 컬럼: {e}")
        return f"데이터 포맷팅 오류: {e}"

def get_korean_stock_info(stock_name: str) -> Optional[str]:
    """
    한국 주식 정보를 조회합니다.
    
    Args:
        stock_name: 조회할 종목명
        
    Returns:
        포맷팅된 주식 정보 문자열 또는 None (오류 시)
    """
    try:
        # 종목 코드 조회
        code = get_stock_code_by_name(stock_name)
        
        # 대상 날짜 계산
        today = datetime.today()
        target_date = get_target_stock_date(today)
        target_str = target_date.strftime("%Y%m%d")
        
        # 주식 데이터 조회
        df = stock.get_market_ohlcv_by_date(
            fromdate=target_str, 
            todate=target_str, 
            ticker=code
        )
        
        if df.empty:
            logger.warning(f"해당 날짜({target_str})에 데이터가 없습니다.")
            return None

        # 데이터 포맷팅 및 반환
        row = df.iloc[0]
        result = format_stock_data(row, stock_name, target_str)
        print(result)
        return result
        
    except ValueError as e:
        logger.error(f"입력 오류: {e}")
        print(f"오류: {e}")
        return None
    except Exception as e:
        logger.error(f"주식 정보 조회 중 예상치 못한 오류: {e}")
        print(f"오류: {e}")
        return None

def clear_stock_cache() -> None:
    """종목 코드 캐시를 초기화합니다."""
    _stock_code_cache.clear()
    logger.info("종목 코드 캐시가 초기화되었습니다.")

# 사용 예시
if __name__ == "__main__":
    # 삼성전자 주식 정보 조회
    result = get_korean_stock_info("SK하이닉스")


[SK하이닉스] 20250620 기준
종가: 257,000.0원
전일대비: 4.47%
거래량: 3,548,848.0주
시가: 249,000.0원
고가: 257,000.0원
저가: 247,500.0원


# 실시간 날씨 예보 - 

In [6]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [10]:
import requests  # 웹 요청을 보내기 위한 라이브러리

# 1. API 키 입력 (회원가입 후 받은 키로 바꾸세요!)
API_KEY = "cb9a8cbc6c0ca6f050057f8b496292fb"  # OpenWeatherMap API 키

# 2. 날씨를 조회할 도시 설정
city = "Seoul"

# 3. API 요청 주소 만들기
url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&lang=kr&units=metric"

# 4. 요청 보내고 응답 받기
response = requests.get(url)

# 5. JSON 형태로 결과 받기
data = response.json()

# 6. 날씨 정보 출력
if response.status_code == 200:
    print(f"도시: {data['name']}")
    print(f"날씨: {data['weather'][0]['description']}")
    print(f"온도: {data['main']['temp']}°C")
    print(f"체감 온도: {data['main']['feels_like']}°C")
    print(f"습도: {data['main']['humidity']}%")
else:
    print("날씨 정보를 가져오지 못했습니다.")
    print("오류 메시지:", data.get("message", "알 수 없는 오류"))


도시: Seoul
날씨: 맑음
온도: 25.76°C
체감 온도: 25.98°C
습도: 61%


In [ ]:
import requests
import json
from datetime import datetime
from typing import Optional, Dict, Any

def get_weather(city: str, country_code: str = "KR") -> Optional[Dict[str, Any]]:
    """
    현재 날씨 정보를 조회합니다.
    
    Args:
        city: 도시명
        country_code: 국가 코드 (기본값: "KR")
        
    Returns:
        날씨 정보 딕셔너리 또는 None (오류 시)
    """
    # API 키 (실제 사용 시 config.json에서 로드하거나 환경변수 사용)
    API_KEY = "cb9a8cbc6c0ca6f050057f8b496292fb"
    
    try:
        # API 요청
        url = f"http://api.openweathermap.org/data/2.5/weather"
        params = {
            "q": f"{city},{country_code}",
            "appid": API_KEY,
            "lang": "kr",
            "units": "metric"
        }
        
        response = requests.get(url, params=params, timeout=10)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"❌ API 오류: {response.status_code}")
            return None
            
    except requests.exceptions.Timeout:
        print("❌ 요청 시간 초과")
        return None
    except requests.exceptions.RequestException as e:
        print(f"❌ 네트워크 오류: {e}")
        return None
    except Exception as e:
        print(f"❌ 예상치 못한 오류: {e}")
        return None

def get_forecast(city: str, country_code: str = "KR", days: int = 5) -> Optional[Dict[str, Any]]:
    """
    날씨 예보 정보를 조회합니다.
    
    Args:
        city: 도시명
        country_code: 국가 코드 (기본값: "KR")
        days: 예보 일수 (기본값: 5)
        
    Returns:
        예보 정보 딕셔너리 또는 None (오류 시)
    """
    API_KEY = "openweathermap_api_key"  # 실제 API 키로 변경하세요
    
    try:
        url = f"http://api.openweathermap.org/data/2.5/forecast"
        params = {
            "q": f"{city},{country_code}",
            "appid": API_KEY,
            "lang": "kr",
            "units": "metric"
        }
        
        response = requests.get(url, params=params, timeout=10)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"❌ API 오류: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"❌ 예보 조회 오류: {e}")
        return None

def format_current_weather(data: Dict[str, Any]) -> str:
    """현재 날씨 정보를 포맷팅"""
    try:
        city = data['name']
        country = data['sys']['country']
        temp = data['main']['temp']
        feels_like = data['main']['feels_like']
        humidity = data['main']['humidity']
        description = data['weather'][0]['description']
        wind_speed = data['wind']['speed']
        
        # 일출/일몰 시간
        sunrise = datetime.fromtimestamp(data['sys']['sunrise']).strftime('%H:%M')
        sunset = datetime.fromtimestamp(data['sys']['sunset']).strftime('%H:%M')
        
        return (
            f"📍 {city}, {country}\n"
            f"🌤️  {description}\n"
            f"🌡️  온도: {temp:.1f}°C\n"
            f"🌡️  체감온도: {feels_like:.1f}°C\n"
            f"💧 습도: {humidity}%\n"
            f"🌪️  바람: {wind_speed:.1f}m/s\n"
            f"🌅 일출: {sunrise}\n"
            f"🌇 일몰: {sunset}"
        )
    except KeyError as e:
        return f"❌ 데이터 파싱 오류: {e}"

def format_forecast(data: Dict[str, Any], days: int = 5) -> str:
    """예보 정보를 포맷팅"""
    try:
        city = data['city']['name']
        result = [f"📅 {city} {days}일 예보"]
        
        # 날짜별로 데이터 그룹화
        daily_data = {}
        for item in data['list']:
            date = datetime.fromtimestamp(item['dt']).strftime('%m/%d')
            if date not in daily_data:
                daily_data[date] = []
            daily_data[date].append(item)
        
        # 각 날짜별 평균 계산
        for i, (date, items) in enumerate(list(daily_data.items())[:days], 1):
            avg_temp = sum(item['main']['temp'] for item in items) / len(items)
            avg_humidity = sum(item['main']['humidity'] for item in items) / len(items)
            description = items[0]['weather'][0]['description']
            
            result.append(
                f"\n📅 {i}일 후 ({date})\n"
                f"🌤️  {description}\n"
                f"🌡️  온도: {avg_temp:.1f}°C\n"
                f"💧 습도: {avg_humidity:.0f}%"
            )
        
        return "\n".join(result)
        
    except KeyError as e:
        return f"❌ 예보 데이터 파싱 오류: {e}"

def main():
    """메인 함수"""
    print("=" * 50)
    print("🌤️  날씨 정보 조회")
    print("=" * 50)
    
    # 현재 날씨 조회
    print("\n📍 현재 날씨")
    current_weather = get_weather("Seoul")
    if current_weather:
        print(format_current_weather(current_weather))
    else:
        print("❌ 현재 날씨 정보를 가져올 수 없습니다.")
    
    # 날씨 예보 조회
    print("\n" + "=" * 50)
    print("📅 5일 예보")
    print("=" * 50)
    
    forecast = get_forecast("Seoul", days=5)
    if forecast:
        print(format_forecast(forecast, 5))
    else:
        print("❌ 날씨 예보를 가져올 수 없습니다.")

if __name__ == "__main__":
    main() 

🌤️  날씨 정보 조회

📍 현재 날씨
📍 Seoul, KR
🌤️  맑음
🌡️  온도: 25.8°C
🌡️  체감온도: 26.0°C
💧 습도: 61%
🌪️  바람: 2.6m/s
🌅 일출: 05:11
🌇 일몰: 19:56

📅 5일 예보
📅 Seoul 5일 예보

📅 1일 후 (06/22)
🌤️  맑음
🌡️  온도: 25.9°C
💧 습도: 56%

📅 2일 후 (06/23)
🌤️  맑음
🌡️  온도: 25.5°C
💧 습도: 55%

📅 3일 후 (06/24)
🌤️  온흐림
🌡️  온도: 25.1°C
💧 습도: 59%

📅 4일 후 (06/25)
🌤️  실 비
🌡️  온도: 21.4°C
💧 습도: 81%

📅 5일 후 (06/26)
🌤️  실 비
🌡️  온도: 21.1°C
💧 습도: 85%


In [26]:
import requests

def get_quote():
    try:
        response = requests.get("https://korean-advice-open-api.vercel.app/api/advice")
        if response.status_code == 200:
            data = response.json()
            return f'"{data["message"]}"\n- {data["author"]}'
        else:
            return "명언을 가져오지 못했습니다."
    except Exception as e:
        return f"오류 발생: {e}"
    
get_quote()    

'"나만이 내 인생을 바꿀 수 있다. 아무도 날 대신해 해줄 수 없다."\n- 캐롤 버넷'

In [ ]:
# plyer install  use pip
%pip install plyer

Note: you may need to restart the kernel to use updated packages.


In [23]:
from plyer import notification

def show_notification(message):
    notification.notify(
        title="오늘의 명언 😊",
        message=message,
        timeout=10  # 초
    )
show_notification( get_quote() )

In [24]:
%pip install schedule

Note: you may need to restart the kernel to use updated packages.


In [25]:
import schedule
import time

def job():
    message = get_quote()
    show_notification(message)

# 매일 오전 9시에 실행
schedule.every().day.at("12:14").do(job)

print("⏰ 알림 대기 중... (Ctrl+C로 종료)")
while True:
    schedule.run_pending()
    time.sleep(1)


⏰ 알림 대기 중... (Ctrl+C로 종료)


KeyboardInterrupt: 

In [ ]:
import requests
from plyer import notification
import schedule
import time

def get_quote():
    try:
        response = requests.get("https://korean-advice-open-api.vercel.app/api/advice")
        if response.status_code == 200:
            data = response.json()
            return f'"{data["content"]}"\n- {data["author"]}'
        else:
            return "명언을 가져오지 못했습니다."
    except Exception as e:
        return f"오류 발생: {e}"

def show_notification(message):
    notification.notify(
        title="오늘의 명언 😊",
        message=message,
        timeout=10
    )

def job():
    message = get_quote()
    show_notification(message)

# 매일 오전 9시 알림
schedule.every().day.at("09:00").do(job)

print("⏰ 알림 대기 중... (Ctrl+C로 종료)")
while True:
    schedule.run_pending()
    time.sleep(1)


In [27]:
import requests
from plyer import notification
import schedule
import time
import json
import logging

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# 설정
CONFIG = {
    "notification_time": "09:00",
    "notification_timeout": 10,
    "notification_title": "오늘의 명언 😊",
    "api_url": "https://korean-advice-open-api.vercel.app/api/advice",
    "max_retries": 3,
    "retry_delay": 5
}

# 명언 캐시
quote_cache = []
MAX_CACHE_SIZE = 10

def load_config():
    """설정 파일을 로드합니다."""
    try:
        with open('quote_config.json', 'r', encoding='utf-8') as f:
            user_config = json.load(f)
            CONFIG.update(user_config)
            logger.info("설정 파일을 성공적으로 로드했습니다.")
    except FileNotFoundError:
        # 기본 설정 파일 생성
        with open('quote_config.json', 'w', encoding='utf-8') as f:
            json.dump(CONFIG, f, ensure_ascii=False, indent=2)
        logger.info("기본 설정 파일을 생성했습니다.")
    except Exception as e:
        logger.error(f"설정 파일 로드 오류: {e}")

def get_quote():
    """
    한국어 명언 API에서 명언을 가져옵니다.
    원본 코드의 get_quote 함수를 개선했습니다.
    """
    for attempt in range(CONFIG["max_retries"]):
        try:
            logger.info(f"명언 요청 시도 {attempt + 1}/{CONFIG['max_retries']}")
            
            response = requests.get(
                CONFIG["api_url"], 
                timeout=10,
                headers={'User-Agent': 'QuoteNotification/1.0'}
            )
            
            if response.status_code == 200:
                data = response.json()
                
                # API 응답 구조 확인 및 처리
                if "content" in data and "author" in data:
                    quote_text = f'"{data["content"]}"\n- {data["author"]}'
                elif "advice" in data:  # 다른 응답 구조 대응
                    quote_text = f'"{data["advice"]}"\n- 한국의 지혜'
                else:
                    # 예상치 못한 응답 구조 처리
                    quote_text = f'"{str(data)}"\n- 알 수 없는 출처'
                
                # 캐시에 추가
                add_to_cache(quote_text)
                
                logger.info(f"명언을 성공적으로 가져왔습니다: {data.get('author', '알 수 없음')}")
                return quote_text
                
            else:
                logger.warning(f"API 응답 오류: {response.status_code}")
                if response.status_code == 404:
                    return "API 엔드포인트를 찾을 수 없습니다."
                elif response.status_code == 500:
                    return "서버 오류가 발생했습니다."
                else:
                    return f"API 오류: {response.status_code}"
                
        except requests.exceptions.Timeout:
            logger.error(f"요청 시간 초과 (시도 {attempt + 1})")
        except requests.exceptions.ConnectionError:
            logger.error(f"연결 오류 (시도 {attempt + 1})")
        except requests.exceptions.RequestException as e:
            logger.error(f"네트워크 오류 (시도 {attempt + 1}): {e}")
        except json.JSONDecodeError as e:
            logger.error(f"JSON 파싱 오류 (시도 {attempt + 1}): {e}")
        except Exception as e:
            logger.error(f"예상치 못한 오류 (시도 {attempt + 1}): {e}")
        
        # 재시도 전 대기
        if attempt < CONFIG["max_retries"] - 1:
            logger.info(f"{CONFIG['retry_delay']}초 후 재시도합니다...")
            time.sleep(CONFIG["retry_delay"])
    
    # 모든 시도 실패 시 캐시에서 랜덤 선택
    if quote_cache:
        logger.warning("API 요청 실패, 캐시된 명언을 사용합니다.")
        import random
        return random.choice(quote_cache)
    
    # 캐시도 없으면 기본 명언 반환
    default_quotes = [
        '"오늘 하루도 힘내세요!"\n- 응원하는 마음',
        '"작은 진전도 진전입니다."\n- 격려의 말',
        '"당신은 충분히 잘하고 있습니다."\n- 따뜻한 위로',
        '"내일은 더 좋은 날이 될 거예요."\n- 희망의 메시지',
        '"한 걸음씩 천천히 나아가세요."\n- 인생의 지혜'
    ]
    import random
    return random.choice(default_quotes)

def add_to_cache(quote):
    """명언을 캐시에 추가합니다."""
    if quote not in quote_cache:
        quote_cache.append(quote)
        if len(quote_cache) > MAX_CACHE_SIZE:
            quote_cache.pop(0)  # 가장 오래된 명언 제거

def show_notification(message):
    """
    윈도우 알림을 표시합니다.
    원본 코드의 show_notification 함수를 개선했습니다.
    """
    try:
        # 원본 코드와 동일한 방식으로 호출
        notification.notify(
            title=CONFIG["notification_title"],
            message=message,
            timeout=CONFIG["notification_timeout"]
        )
        logger.info("알림을 성공적으로 표시했습니다.")
        
    except Exception as e:
        logger.error(f"알림 표시 오류: {e}")
        # 알림 실패 시 콘솔에 출력
        print(f"\n{'='*50}")
        print(f"📝 {CONFIG['notification_title']}")
        print(f"{'='*50}")
        print(message)
        print(f"{'='*50}\n")

def job():
    """
    스케줄된 작업을 실행합니다.
    원본 코드의 job 함수를 개선했습니다.
    """
    logger.info("명언 알림 작업을 시작합니다.")
    
    message = get_quote()
    show_notification(message)

def main():
    """메인 함수"""
    try:
        # 설정 로드
        load_config()
        
        # 스케줄 설정 (원본 코드와 동일)
        schedule.every().day.at(CONFIG["notification_time"]).do(job)
        
        logger.info(f"스케줄러가 시작되었습니다. 알림 시간: {CONFIG['notification_time']}")
        print(f"⏰ 알림 대기 중... (매일 {CONFIG['notification_time']})")
        print("📝 Ctrl+C로 종료")
        print("🔧 설정 변경 시 quote_config.json 파일을 수정하세요")
        print(f"🌐 API: {CONFIG['api_url']}")
        
        # 즉시 한 번 실행 (테스트용)
        print("\n🧪 테스트 알림을 보내시겠습니까? (y/n): ", end="")
        if input().lower() == 'y':
            print("🧪 테스트 알림을 실행합니다...")
            job()
            print("✅ 테스트 알림이 완료되었습니다.")
        
        # 원본 코드와 동일한 스케줄러 루프
        print("\n⏰ 알림 대기 중... (Ctrl+C로 종료)")
        while True:
            schedule.run_pending()
            time.sleep(1)
            
    except KeyboardInterrupt:
        logger.info("사용자에 의해 프로그램이 종료되었습니다.")
        print("\n👋 프로그램을 종료합니다.")
    except Exception as e:
        logger.error(f"프로그램 실행 오류: {e}")
        print(f"❌ 오류가 발생했습니다: {e}")

if __name__ == "__main__":
    main() 

INFO:__main__:기본 설정 파일을 생성했습니다.
INFO:__main__:스케줄러가 시작되었습니다. 알림 시간: 09:00


⏰ 알림 대기 중... (매일 09:00)
📝 Ctrl+C로 종료
🔧 설정 변경 시 quote_config.json 파일을 수정하세요
🌐 API: https://korean-advice-open-api.vercel.app/api/advice

🧪 테스트 알림을 보내시겠습니까? (y/n): 

INFO:__main__:명언 알림 작업을 시작합니다.
INFO:__main__:명언 요청 시도 1/3


🧪 테스트 알림을 실행합니다...


INFO:__main__:명언을 성공적으로 가져왔습니다: 스탕달
INFO:__main__:알림을 성공적으로 표시했습니다.


✅ 테스트 알림이 완료되었습니다.

⏰ 알림 대기 중... (Ctrl+C로 종료)


INFO:__main__:사용자에 의해 프로그램이 종료되었습니다.



👋 프로그램을 종료합니다.
